In [ ]:
import socket
import threading
import os

class EchoServerThread(threading.Thread):
    def __init__(self, client_socket, client_address):
        threading.Thread.__init__(self)
        self.client_socket = client_socket
        self.client_address = client_address
        print(f"Connection from {client_address}")

    def run(self):
        try:
            # Informer le client de la connexion
            self.client_socket.sendall("Client connected".encode('utf-8'))

            # Recevoir le nom du client
            nom = self.client_socket.recv(1024).decode('utf-8').strip()
            print(f"Client name: {nom}")

            # Demander au client s'il veut uploader ou télécharger un fichier
            self.client_socket.sendall("0 pour télécharger ou 1 pour uploader".encode('utf-8'))
            choice = self.client_socket.recv(1024).decode('utf-8').strip()

            if choice == "0":
                self.send_file()
            elif choice == "1":
                self.receive_file()
            else:
                self.client_socket.sendall("Choix invalide".encode('utf-8'))
                print(f"Client {nom} a fait un choix invalide.")

        except Exception as e:
            print(f"Error: {e}")

        finally:
            self.client_socket.close()
            print(f"Connection with {self.client_address} closed")

    def send_file(self):
        filename = self.client_socket.recv(1024).decode('utf-8').strip()
        if os.path.exists(filename):
            self.client_socket.sendall(f"ENVOI {filename}".encode('utf-8'))
            with open(filename, 'rb') as file:
                while True:
                    data = file.read(1024)
                    if not data:
                        break
                    self.client_socket.sendall(data)
            print(f"File {filename} sent successfully.")
        else:
            self.client_socket.sendall("Le fichier n'existe pas".encode('utf-8'))

    def receive_file(self):
        filename = self.client_socket.recv(1024).decode('utf-8').strip()
        self.client_socket.sendall(f"READY TO RECEIVE {filename}".encode('utf-8'))
        with open(filename, 'wb') as file:
            while True:
                data = self.client_socket.recv(1024)
                if not data:
                    break
                file.write(data)
        print(f"File {filename} received successfully.")

# Définir le numéro de port du serveur
port_number = 9000

# Indiquer que le serveur écoute les informations arrivant sur son port
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', port_number))
server_socket.listen(1)

print(f"Server is listening on port {port_number}")

while True:
    client_socket, client_address = server_socket.accept()
    EchoServerThread(client_socket, client_address).start()



Server is listening on port 9000
